In [1]:
import numpy as np
import cv2
import open3d as o3d
import open3d.visualization.rendering as rendering

from utils.meta_io import fetch_scene_object_by_image_id
from utils.intrinsic_fetcher import IntrinsicFetcher

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
scene_by_image_id = fetch_scene_object_by_image_id('v2_3d')
intrinsic_fetcher = IntrinsicFetcher()

In [18]:
render_to_image('000001_1_0')

TypeError: add_geometry(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cuda.pybind.visualization.rendering.Open3DScene, name: str, geometry: open3d.cuda.pybind.geometry.Geometry3D, material: open3d.cuda.pybind.visualization.rendering.MaterialRecord, add_downsampled_copy_for_fast_rendering: bool = True) -> None
    2. (self: open3d.cuda.pybind.visualization.rendering.Open3DScene, name: str, geometry: open3d.cuda.pybind.t.geometry.Geometry, material: open3d.cuda.pybind.visualization.rendering.MaterialRecord, add_downsampled_copy_for_fast_rendering: bool = True) -> None

Invoked with: <open3d.cuda.pybind.visualization.rendering.Open3DScene object at 0x7f598b9832b0>, 'pcd', PointCloud with 280687 points.

In [17]:
def render_to_image(uniq_id: str):
    image_id, object_id, anno_id = uniq_id.split('_')
    object_id = int(object_id)
    scene = scene_by_image_id[image_id]
    fx, fy, cx, cy, height, width = intrinsic_fetcher[image_id]

    render = rendering.OffscreenRenderer(width, height)
    color_raw = cv2.imread(str(scene.rgb_path), cv2.IMREAD_COLOR)
    depth_raw = cv2.imread(str(scene.depth_path), cv2.IMREAD_UNCHANGED)
    rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(
        o3d.geometry.Image(color_raw),
        o3d.geometry.Image(depth_raw),
        convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        image=rgbd_image,
        intrinsic=o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy))
    color = rendering.MaterialRecord()
    color.base_color = [0, 0, 0, 1]
    color.shader = 'defaultLit'
    render.scene.add_geometry('pcd', pcd, color)

    Rt = np.eye(4, dtype=np.float32)
    Rt[:3, :3] = scene.extrinsics
    render.setup_camera(o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy), Rt)
    img = render.render_to_image()
    o3d.io.write_image('test.png', img, 9)

    # Tyz = np.array([
    #     [1, 0, 0, 0],
    #     [0, 0, 1, 0],
    #     [0, -1, 0, 0],
    #     [0, 0, 0, 1]], dtype=np.float32)
    # Rt = Tyz @ Rt




    # o3d_obj_list = []
    # o3d_obj_list.append(pcd)
    #
    # coord_frame_orig = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
    # # coord_frame_revised = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
    # # coord_frame_revised.transform(np.linalg.inv(Rt))
    # #
    # o3d_obj_list.append(coord_frame_orig)
    # # o3d_obj_list.append(coord_frame_revised)
    # #
    # # for oid, bbox_3d in enumerate(scene.gt_3d_bbox):
    # #     # class_name = bbox_3d.class_name[0]  # str
    # #     centroid = bbox_3d.centroid[0]  # (3, )
    # #     basis = bbox_3d.basis  # (3, 3)
    # #     coeffs = bbox_3d.coeffs[0]  # (3, )
    # #
    # #     if self.highlight and oid == object_id:
    # #         color = [0, 1, 0]
    # #         radius = 0.03
    # #     else:
    # #         color = [1, 0, 0]
    # #         radius = 0.015
    # #
    # #     line_mesh = create_line_mesh(centroid, basis, coeffs, aabb=self.aabb, color=color, radius=radius)
    # #     for s in line_mesh.cylinder_segments:
    # #         s.transform(np.linalg.inv(Rt))
    # #     o3d_obj_list += line_mesh.cylinder_segments
    # o3d.visualization.draw_geometries(o3d_obj_list)a

In [61]:
def compute_3d_bbox(uniq_id: str):
    image_id, object_id, anno_id = uniq_id.split('_')
    object_id = int(object_id)
    scene = scene_by_image_id[image_id]
    fx, fy, cx, cy, height, width = intrinsic_fetcher[image_id]

    Rt = np.eye(4, dtype=np.float32)
    Rt[:3, :3] = scene.extrinsics

    print(scene.extrinsics)

    # Tyz = np.array([
    #     [1, 0, 0, 0],
    #     [0, 0, 1, 0],
    #     [0, -1, 0, 0],
    #     [0, 0, 0, 1]], dtype=np.float32)
    # Rt = Tyz @ Rt

    # Rt = np.array([
    #     [1, 0, 0, 0],
    #     [0, 0, 1, 0],
    #     [0, -1, 0, 0],
    #     [0, 0, 0, 1]], dtype=np.float32)

    print(Rt)
    print(',\n'.join(list(map(str, np.reshape(np.linalg.inv(Rt), (16, )).tolist()))))

    color_raw = cv2.imread(str(scene.rgb_path), cv2.IMREAD_COLOR)
    color_raw = cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB)
    depth_raw = cv2.imread(str(scene.depth_path), cv2.IMREAD_UNCHANGED)
    rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(
        o3d.geometry.Image(color_raw),
        o3d.geometry.Image(depth_raw),
        convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        image=rgbd_image,
        intrinsic=o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy))
        # extrinsic=Rt)
    o3d.io.write_point_cloud("pcd.ply", pcd)

    # o3d_obj_list = []
    # o3d_obj_list.append(pcd)

    coord_frame_orig = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
    viewer = o3d.visualization.Visualizer()
    viewer.create_window('viewer', visible=True, width=width, height=height)
    viewer.add_geometry(pcd)
    viewer.add_geometry(coord_frame_orig)
    cam = viewer.get_view_control().convert_to_pinhole_camera_parameters()
    cam.intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)
    print(cam.intrinsic.intrinsic_matrix)
    # print(cam.intrinsic)
    # cam.extrinsic = Rt  #np.linalg.inv(Rt)
    # print(cam)
    viewer.get_view_control().convert_from_pinhole_camera_parameters(cam)
    viewer.run()
    viewer.capture_screen_image('screen.png')
    viewer.destroy_window()

    # o3d.visualization.draw_geometries(o3d_obj_list)

In [62]:
compute_3d_bbox('000001_1_0')

[[ 0.979589  0.200614  0.012593]
 [-0.200614  0.97182   0.123772]
 [ 0.012593 -0.123772  0.992231]]
[[ 0.979589  0.200614  0.012593  0.      ]
 [-0.200614  0.97182   0.123772  0.      ]
 [ 0.012593 -0.123772  0.992231  0.      ]
 [ 0.        0.        0.        1.      ]]
0.9795898199081421,
-0.20061418414115906,
0.012592272832989693,
0.0,
0.20061418414115906,
0.9718203544616699,
-0.12377206981182098,
0.0,
0.012592272832989693,
0.12377206981182098,
0.9922305345535278,
0.0,
0.0,
0.0,
0.0,
1.0
[[529.5   0.  365. ]
 [  0.  529.5 265. ]
 [  0.    0.    1. ]]
[Open3D WARNING] [ViewControl] ConvertFromPinholeCameraParameters() failed because window height and width do not match.


In [68]:
def convert_raw_depth(raw_depth):
    """Numpy implementation of SUNRGBD depth processing."""
    shift_depth = np.bitwise_or(np.right_shift(raw_depth, 3), np.left_shift(raw_depth, 13))
    float_depth = shift_depth.astype(dtype=np.float32) / 1000.
    float_depth[float_depth > 7.0] = 0.
    return float_depth


def convert_pcd(raw_depth, fx, fy, cx, cy):
    m = raw_depth > 0
    zz = convert_raw_depth(raw_depth)
    mx = np.linspace(0, raw_depth.shape[1] - 1, raw_depth.shape[1])
    my = np.linspace(0, raw_depth.shape[0] - 1, raw_depth.shape[0])
    rx = np.linspace(0., 1., raw_depth.shape[1])
    ry = np.linspace(0., 1., raw_depth.shape[0])
    rx, ry = np.meshgrid(rx, ry)
    xx, yy = np.meshgrid(mx, my)
    xx = (xx - cx) * zz / fx
    yy = (yy - cy) * zz / fy
    pcd = np.stack((xx, yy, zz), axis=-1)
    return pcd, m, rx, ry


def interpolate_projected_points(uniq_id: str):
    image_id, object_id, anno_id = uniq_id.split('_')
    object_id = int(object_id)
    scene = scene_by_image_id[image_id]
    fx, fy, cx, cy, height, width = intrinsic_fetcher[image_id]

    color_raw = cv2.imread(str(scene.rgb_path), cv2.IMREAD_COLOR)
    color_raw = cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB)
    depth_raw = cv2.imread(str(scene.depth_path), cv2.IMREAD_UNCHANGED)
    color_np = np.asarray(color_raw, dtype=np.uint8)
    depth_np = np.asarray(depth_raw, dtype=np.uint16)
    pcd, mask, rx, ry = convert_pcd(raw_depth=depth_np, fx=fx, fy=fy, cx=cx, cy=cy)

    Rt = np.array(scene.extrinsics)
    # Rt = np.array([
    #     [1, 0, 0],
    #     [0, 0, 1],
    #     [0, -1, 0]], dtype=np.float32)
    # Rt = Tyz @ Rt

    # print(Rt)
    # print(np.linalg.inv(Rt))
    pcd = pcd @ np.transpose(Rt)
    # pcd = pcd @ np.linalg.inv(Rt)

    print(Rt.shape)
    print(pcd.shape)
    print(mask.shape)

    pcd[mask, 0] *= fx / pcd[mask, 2]
    pcd[mask, 1] *= fy / pcd[mask, 2]

    new_image = np.zeros((height, width, 3), dtype=np.uint8)



    # reproject to 2D image again after transformation.
    from math import isnan

    for rr in range(pcd.shape[0]):
        for cc in range(pcd.shape[1]):
            if isnan(pcd[rr, cc, 0]) or isnan(pcd[rr, cc, 1]):
                continue
            xx = int(pcd[rr, cc, 0] + cx)
            yy = int(pcd[rr, cc, 1] + cy)
            # if rr == 0:
            #     print(rr, nr, cc, nc)
            if 0 <= xx < pcd.shape[1] and 0 <= yy < pcd.shape[0]:
                new_image[yy, xx, :] = color_np[rr, cc, :]

    # for i, rr in enumerate(range(pcd.shape[0])):
    #     for j, cc in enumerate(range(pcd.shape[1])):
    #         ir, ic = int(rr), int(cc)
    #         if 0 <= ir < pcd.shape[1] and 0 <= ic < pcd.shape[0]:
    #             new_image[ir, ic, :] = color_np[i, j, :]

    mask = mask.astype(np.uint8) * 255

    o3d.io.write_image('test.png', o3d.geometry.Image(new_image))
    o3d.io.write_image('mask.png', o3d.geometry.Image(mask))



    # image = np.zeros((height, width, 3), dtype=np.uint8)

    # Rt = np.eye(4, dtype=np.float32)
    # Rt[:3, :3] = scene.extrinsics

    # rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(
    #     o3d.geometry.Image(color_raw),
    #     o3d.geometry.Image(depth_raw),
    #     convert_rgb_to_intensity=False)
    # pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    #     image=rgbd_image,
    #     intrinsic=o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy))
        # extrinsic=Rt)
    # o3d.io.write_point_cloud("pcd.ply", pcd)

    # o3d_obj_list = []
    # o3d_obj_list.append(pcd)

    # coord_frame_orig = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 0, 0])
    # viewer = o3d.visualization.Visualizer()
    # viewer.create_window('viewer', visible=True, width=width, height=height)
    # viewer.add_geometry(pcd)
    # viewer.add_geometry(coord_frame_orig)
    # cam = viewer.get_view_control().convert_to_pinhole_camera_parameters()
    # cam.intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)
    # print(cam.intrinsic.intrinsic_matrix)
    # # print(cam.intrinsic)
    # # cam.extrinsic = Rt  #np.linalg.inv(Rt)
    # # print(cam)
    # viewer.get_view_control().convert_from_pinhole_camera_parameters(cam)
    # viewer.run()
    # viewer.capture_screen_image('screen.png')
    # viewer.destroy_window()

    # o3d.visualization.draw_geometries(o3d_obj_list)

In [69]:
interpolate_projected_points('000001_1_0')

(3, 3)
(530, 730, 3)
(530, 730)


/tmp/ipykernel_13565/1678654989.py:53: RuntimeWarning: divide by zero encountered in true_divide
  pcd[mask, 0] *= fx / pcd[mask, 2]
/tmp/ipykernel_13565/1678654989.py:53: RuntimeWarning: invalid value encountered in multiply
  pcd[mask, 0] *= fx / pcd[mask, 2]
/tmp/ipykernel_13565/1678654989.py:54: RuntimeWarning: divide by zero encountered in true_divide
  pcd[mask, 1] *= fy / pcd[mask, 2]
/tmp/ipykernel_13565/1678654989.py:54: RuntimeWarning: invalid value encountered in multiply
  pcd[mask, 1] *= fy / pcd[mask, 2]
